# wav2vec-u CV-sv - w2vu_generate
> "w2vu_generate on Common Voice Swedish"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [colab, wav2vec-u]

The original attempt on [Kaggle](https://www.kaggle.com/jimregan/wav2vec-u-cv-swedish-gan) won't run because of an issue with CuDNN, but this notebook runs fine on Colab.

## Preparation

In [1]:
!pip install condacolab

  Using cached condacolab-0.1.2-py3-none-any.whl (6.0 kB)


In [2]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
%%capture
!conda install -c pykaldi pykaldi -y

In [4]:
!git clone https://github.com/pytorch/fairseq/

fatal: destination path 'fairseq' already exists and is not an empty directory.


In [5]:
!git clone https://github.com/kpu/kenlm

fatal: destination path 'kenlm' already exists and is not an empty directory.


In [6]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

The python build doesn't build utils, so this is (probably) necessary

In [67]:
%cd /content/kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4

/content/kenlm
mkdir: cannot create directory ‘build’: File exists
/content/kenlm/build
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
--

In [7]:
%%capture
%cd /content/kenlm
!python setup.py install
%cd /tmp

In [8]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/content/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/content/fairseq'

In [31]:
%cd /content/fairseq/

/content/fairseq


For next cell, see [here](https://github.com/pytorch/fairseq/issues/3087)

In [32]:
%%capture
!pip install --editable ./
!python setup.py build develop

In [11]:
os.environ['HYDRA_FULL_ERROR'] = '1'

In [12]:
%%capture
!pip install editdistance

https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb

In [13]:
%%capture
!pip install kaggle

In [14]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes


In [15]:
%cd /content

/content


In [18]:
!kaggle datasets download "jimregan/w2vu-cvsv-audio-processed"

100% 4.31G/4.31G [01:24<00:00, 102MB/s] 
100% 4.31G/4.31G [01:24<00:00, 54.7MB/s]


In [19]:
%%capture
!unzip /content/w2vu-cvsv-audio-processed.zip

In [20]:
!kaggle datasets download -d jimregan/w2vu-cvsv-prepared-text

 80% 14.0M/17.4M [00:00<00:00, 33.7MB/s]
100% 17.4M/17.4M [00:00<00:00, 44.2MB/s]


In [21]:
%%capture
!unzip w2vu-cvsv-prepared-text.zip

In [22]:
!rm *.zip

In [43]:
!cp /content/preppedtext/phones/dict* /content/precompute_pca512_cls128_mean

In [68]:
%cd /content

/content


In [51]:
!git clone https://github.com/flashlight/flashlight

Cloning into 'flashlight'...
remote: Enumerating objects: 17649, done.
remote: Counting objects: 100% (1523/1523), done.
remote: Compressing objects: 100% (718/718), done.
remote: Total 17649 (delta 827), reused 1336 (delta 761), pack-reused 16126
Receiving objects: 100% (17649/17649), 14.23 MiB | 24.82 MiB/s, done.
Resolving deltas: 100% (12298/12298), done.


In [54]:
%%capture
!apt install -q libfftw3-dev

Reading package lists...
Building dependency tree...
Reading state information...
libfftw3-dev is already the newest version (3.3.7-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [69]:
%cd flashlight/bindings/python

/content/flashlight/bindings/python


In [57]:
%%capture
!pip install packaging

     |████████████████████████████████| 40 kB 5.6 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 


In [70]:
!USE_MKL=0 KENLM_ROOT=/content/kenlm python setup.py install

running install
running bdist_egg
running egg_info
writing flashlight.egg-info/PKG-INFO
writing dependency_links to flashlight.egg-info/dependency_links.txt
writing top-level names to flashlight.egg-info/top_level.txt
package init file 'flashlight/__init__.py' not found (or not a regular file)
package init file 'flashlight/lib/audio/__init__.py' not found (or not a regular file)
package init file 'flashlight/lib/sequence/__init__.py' not found (or not a regular file)
package init file 'flashlight/lib/text/__init__.py' not found (or not a regular file)
reading manifest file 'flashlight.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'flashlight.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
-- -rdynamic supported.
-- CUDA found (library: /usr/local/cuda/lib64/libcudart_static.a;-lpthread;dl;/usr/lib/x86_64-linux-gnu/librt.so include: /usr/local/cuda/include)
-- CUDA 

In [59]:
!cat /content/flashlight/bindings/python/build/temp.linux-x86_64-3.7/CMakeFiles/CMakeError.log

Determining if the pthread_create exist failed with the following output:
Change Dir: /content/flashlight/bindings/python/build/temp.linux-x86_64-3.7/CMakeFiles/CMakeTmp

Run Build Command:"/usr/bin/make" "cmTC_64926/fast"
/usr/bin/make -f CMakeFiles/cmTC_64926.dir/build.make CMakeFiles/cmTC_64926.dir/build
make[1]: Entering directory '/content/flashlight/bindings/python/build/temp.linux-x86_64-3.7/CMakeFiles/CMakeTmp'
Building C object CMakeFiles/cmTC_64926.dir/CheckSymbolExists.c.o
/usr/bin/cc   -fPIC    -o CMakeFiles/cmTC_64926.dir/CheckSymbolExists.c.o   -c /content/flashlight/bindings/python/build/temp.linux-x86_64-3.7/CMakeFiles/CMakeTmp/CheckSymbolExists.c
Linking C executable cmTC_64926
/usr/local/lib/python2.7/dist-packages/cmake/data/bin/cmake -E cmake_link_script CMakeFiles/cmTC_64926.dir/link.txt --verbose=1
/usr/bin/cc -fPIC    -rdynamic  CMakeFiles/cmTC_64926.dir/CheckSymbolExists.c.o  -o cmTC_64926 
CMakeFiles/cmTC_64926.dir/CheckSymbolExists.c.o: In function `main':
Che

## GAN

In [23]:
import torch
torch.version.cuda

'10.2'

In [24]:
torch.backends.cudnn.version()

7605

In [25]:
%cd /content/fairseq

/content/fairseq


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [72]:
%cd /content/fairseq/examples/wav2vec/unsupervised

/content/fairseq/examples/wav2vec/unsupervised


In [47]:
%%writefile rungan.sh
python w2vu_generate.py --config-dir config/generate --config-name viterbi \
fairseq.common.user_dir=/content/fairseq/examples/wav2vec/unsupervised \
fairseq.task.data=/content/precompute_pca512_cls128_mean \
fairseq.common_eval.path=/content/drive/MyDrive/w2vu/checkpoint_best.pt \
fairseq.dataset.gen_subset=valid results_path=/content/drive/MyDrive/w2vures

Overwriting rungan.sh


ERROR: Could not find a version that satisfies the requirement pyflashlight
ERROR: No matching distribution found for pyflashlight


In [73]:
!bash rungan.sh

[2021-06-19 19:24:18,601][__main__][INFO] - {'_name': None, 'fairseq': {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/content/fairseq/examples/wav2vec/unsupervised', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 